In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from tqdm import tqdm
from copy import deepcopy
import torch.nn.functional as F



class FeatureHook:
    """Intercepte les activations d'une couche specifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        # Pour les transformers, output peut etre un tuple
        if isinstance(output, tuple):
            self.features = output[0]
        else:
            self.features = output

    def close(self):
        self.hook.remove()


class GreedySentiment:
    def __init__(self, model_id="prajjwal1/bert-tiny", device="cuda"):
        self.device = device
        self.model_id = model_id
        
        # Chargement du modele (bert-tiny: 4.4M params, ~15x plus rapide que DistilBERT)
        # self.model = AutoModelForSequenceClassification.from_pretrained(
        #     model_id, num_labels=2, use_safetensors=True
        # ).to(device)

        config = BertConfig.from_pretrained(model_id, num_labels=2)
        self.model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
        self.tokenizer=BertTokenizer.from_pretrained(model_id)
        # self.tokenizer = AutoTokenizer.from_pretrained(model_id)
        
        # Configuration par defaut
        self.config = {
            # "layer_name": "classifier.weight",  # Couche de classification
             "layer_name": "bert.encoder.layer.1.output.dense.weight",
            "hook_layer":  "bert.encoder.layer.1.output.dense",

            "watermark_size": 16,
            "lr": 5e-5,  # Learning rate plus eleve pour petit modele
            "lambda_wat": 1.0,
            "epochs": 30,  # Plus d'epochs car modele non-pretrained sur sentiment
            "objective_size": (16, 16),     # (watermark_bits, weights_per_bit)
            "ratio": 0.5,                     # Ratio de pruning
            "threshold": 0.1,                 # Seuil pour la loss
            "lambda_wat": 0.01,
            "message": "Transformer_WATERMARKing",
        }
        
        self.saved_keys = {}
        
    def list_layers(self):
        """Affiche les couches disponibles pour le watermarking."""
        print("Couches disponibles:")
        for name, param in self.model.named_parameters():
            print(f"  {name}: {param.shape}")

    def _get_target_layer(self, model, layer_name=None):
        """Recupere la couche cible pour le hook."""

        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")



    def _get_target_weights(self, model):
        """Recupere le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Parametre {self.config['layer_name']} introuvable.")


    def keygen(self, message):
        """
        Génère un watermark aléatoire basé sur le message.
        Returns:
            watermark_sig: Watermark en {-1, +1}
            watermark: Watermark binaire en {0, 1}
        """
        watermark_size = self.config.get("watermark_size", 32)
        print(watermark_size)

        # Set seed based on message for reproducibility
        seed = hash(message) % (2**32)
        torch.manual_seed(seed)

        # Generate random binary watermark {0, 1}
        b_sig = torch.randint(0, 2, (watermark_size,)).tolist()

        # Convert to signs: 0 -> -1, 1 -> +1
        watermark_sig = torch.tensor(
            [-1.0 if i == 0 else 1.0 for i in b_sig],
            dtype=torch.float
        ).to(self.device)

        # Binary watermark {0, 1}
        watermark = torch.tensor(b_sig, dtype=torch.float32).to(self.device)

        return watermark_sig, watermark

    def weight_extraction(self, model):
        """
        Extrait et reshape les poids de la couche cible.
        Returns:
            Tensor de taille objective_size
        """
        target_param = self._get_target_weights(model)

        # Flatten les poids
        layer_weight = target_param.reshape(-1)

        # Tronquer pour être divisible par objective_size[1]
        obj_size = self.config['objective_size']
        total_needed = obj_size[0] * obj_size[1]

        if layer_weight.numel() < total_needed:
            raise ValueError(
                f"Layer has {layer_weight.numel()} weights, "
                f"but objective_size requires {total_needed}"
            )

        # Adaptive average pooling pour obtenir la bonne taille
        truncated = layer_weight[:layer_weight.numel() // obj_size[1] * obj_size[1]]
        pooled = F.adaptive_avg_pool1d(
            truncated[None, None],
            obj_size[0] * obj_size[1]
        ).squeeze(0).view(obj_size)

        return pooled



    def compute_residuals(self, weight_matrix):
        """Vectorized version - much faster."""
        num_rows, num_cols = weight_matrix.shape
        num_to_prune = int(num_cols * self.config['ratio'] + 0.5)

        weights_copy = weight_matrix.clone()

        # Vectorized pruning - no for loop!
        sorted_indices = torch.argsort(torch.abs(weights_copy), dim=1)

        # Create mask for weights to zero out
        mask = torch.ones_like(weights_copy, dtype=torch.bool)
        rows = torch.arange(num_rows, device=weights_copy.device).unsqueeze(1)
        mask[rows, sorted_indices[:, :num_to_prune]] = False

        # Apply mask
        weights_copy = weights_copy * mask

        # Mean of each row
        residual_vector = torch.mean(weights_copy, dim=1)

        return residual_vector



    @staticmethod
    def _compute_ber(extracted, target):
        """Calcule le Bit Error Rate."""
        return (extracted != target).float().mean().item()

    
    def embed(self, train_dataloader, eval_dataloader=None):
        """
        Incorpore la marque greedy pendant le finetuning.
        """
        print(f"--- Demarrage Embedding Greedy  ({self.config['layer_name']}) ---")

        print(f"Message: {self.config['message']}")
        print(f"Objective size: {self.config['objective_size']}")
        print(f"Pruning ratio: {self.config['ratio']}")
        
        watermarked_model = self.model
        watermarked_model.train()


        # 1. Génération des clés (watermark de 256 bits)
        watermark_sig, watermark = self.keygen(self.config["message"])
        watermark_size = self.config.get("watermark_size",16)
        print(watermark_size)
        if self.config['objective_size'][0] != watermark_size:
            print(f"⚠️ Adjusting objective_size[0] to {watermark_size} (watermark size)")
            self.config['objective_size'] = (watermark_size, self.config['objective_size'][1])
        print(f"Watermark size: {len(watermark)} bits")

        # 2. Vérification initiale
        weight_extract = self.weight_extraction(watermarked_model)
        residual_vector = self.compute_residuals(weight_extract)
        print(f"Initial residual vector shape: {residual_vector.shape}")



        # 2. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_model.parameters(), lr=self.config["lr"])
        ce_loss = nn.CrossEntropyLoss()
        bce_loss = nn.BCELoss()
        
        # 3. Boucle d'entrainement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            total_loss = 0
            correct = 0
            total = 0
            
            for batch in pbar:
                input_ids = batch["input_ids"].to(self.device)  #tokennized textes (numbers)
                attention_mask = batch["attention_mask"].to(self.device) #which tokens to pay attention to (1 for real tokens, 0 for padding)
                labels = batch["labels"].to(self.device) #correct answer for each sentence ( 0 = negative, 1 = positive)
                
                optimizer.zero_grad()
                
                # A. Forward pass (Task Loss)
                outputs = watermarked_model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                l_main = ce_loss(logits, labels)
                
                # B. Watermark Loss (Regularization)
                weight_extract = self.weight_extraction(watermarked_model)
                residual_vector = self.compute_residuals(weight_extract)

                l_wat = F.relu(
                    self.config['threshold'] -
                    (watermark_sig * residual_vector)
                ).sum()




                
                # Loss Totale
                l_total = l_main + self.config["lambda_wat"] * l_wat
                
                l_total.backward()
                optimizer.step()
                
                # Metrics
                total_loss += l_main.item()
                preds = torch.argmax(logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)

                ber,_= self.extract(watermarked_model, watermark)

                acc = 100. * correct / total
                pbar.set_postfix(L_Main=f"{l_main.item():.3f}", L_Wat=f"{l_wat.item():.3f}", 
                                BER=f"{ber:.2f}", Acc=f"{acc:.1f}%")
            
            # Evaluation en fin d'epoch
            if eval_dataloader:
                eval_acc = self.evaluate(watermarked_model, eval_dataloader)
                print(f"Epoch {epoch+1} | Eval Accuracy: {eval_acc:.2f}%")
        
        # Sauvegarde des cles
        self.saved_keys = {
            "watermark": watermark,
            "watermark_sig": watermark_sig,
            "watermarked_model": watermarked_model
        }
        
        final_ber,_ =  self.extract(watermarked_model)
        print(f"\n--- Embedding termine | BER final: {final_ber:.2f} ---")
        return watermarked_model
    
    def evaluate(self, model, dataloader):
        """Evalue la precision du modele."""
        model.eval()
        correct = 0
        total = 0
        
        with torch.no_grad():
            for batch in dataloader:
                input_ids = batch["input_ids"].to(self.device)
                attention_mask = batch["attention_mask"].to(self.device)
                labels = batch["labels"].to(self.device)
                
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                preds = torch.argmax(outputs.logits, dim=-1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        
        model.train()
        return 100. * correct / total
    
    def extract(self, suspect_model=None, watermark=None):
        """
        Extrait la marque d'un modele suspect.
        """
        if suspect_model is None:
            suspect_model = self.saved_keys["watermarked_model"]

        if watermark is None:
               watermark = self.saved_keys["watermark"]

        
        # 1. Recuperation des poids
        with torch.no_grad():
            try:
                weight_extract = self.weight_extraction(suspect_model)
                residual_vector = self.compute_residuals(weight_extract)
                signs = torch.sign(residual_vector)
                extracted_wm = signs.clone()
                extracted_wm[extracted_wm == -1] = 0
                ber = self._compute_ber(extracted_wm, watermark)
            except ValueError:
                print(f"⚠️ Erreur lors de l'extraction: {e}")
                return 1.0, None
        

        
        # print(f"BER Extrait : {ber:.2f}")
        return ber, extracted_wm
    
    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()

print("Classe GreedySentiment chargee (bert-tiny: 4.4M params)")

/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Classe GreedySentiment chargee (bert-tiny: 4.4M params)


In [2]:
# --- Chargement des donnees SST-2 ---

# Charger le dataset SST-2 (sentiment binaire)
dataset = load_dataset("glue", "sst2")

# Initialiser le watermarker (charge le modele bert-tiny)
greedy = GreedySentiment(device="cuda" if torch.cuda.is_available() else "cpu")

# Afficher les couches disponibles
greedy.list_layers()

# Tokenization
def tokenize_function(examples):
    return greedy.tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True) #converts raw text sentences into input_ids and attention_mask
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# DataLoaders (batch_size=64 possible grace au petit modele)
train_dataloader = DataLoader(tokenized_dataset["train"], batch_size=64, shuffle=True)
eval_dataloader = DataLoader(tokenized_dataset["validation"], batch_size=64)

print(f"\nTrain samples: {len(tokenized_dataset['train'])}")
print(f"Eval samples: {len(tokenized_dataset['validation'])}")

Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 1948.33it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor

Couches disponibles:
  bert.embeddings.word_embeddings.weight: torch.Size([30522, 128])
  bert.embeddings.position_embeddings.weight: torch.Size([512, 128])
  bert.embeddings.token_type_embeddings.weight: torch.Size([2, 128])
  bert.embeddings.LayerNorm.weight: torch.Size([128])
  bert.embeddings.LayerNorm.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.query.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.query.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.key.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.key.bias: torch.Size([128])
  bert.encoder.layer.0.attention.self.value.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.self.value.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.dense.weight: torch.Size([128, 128])
  bert.encoder.layer.0.attention.output.dense.bias: torch.Size([128])
  bert.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([128])
  bert.encoder.layer

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 34017.26 examples/s]


Train samples: 67349
Eval samples: 872


In [3]:

# Lancer l'embedding
watermarked_model = greedy.embed(train_dataloader, eval_dataloader)

--- Demarrage Embedding Greedy  (bert.encoder.layer.1.output.dense.weight) ---
Message: Transformer_WATERMARKing
Objective size: (16, 16)
Pruning ratio: 0.5
16
16
Watermark size: 16 bits
Initial residual vector shape: torch.Size([16])


Epoch 1/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 213.41it/s, Acc=79.5%, BER=0.31, L_Main=0.258, L_Wat=1.596]


Epoch 1 | Eval Accuracy: 79.70%


Epoch 2/30: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 219.21it/s, Acc=88.4%, BER=0.25, L_Main=0.383, L_Wat=1.590]


Epoch 2 | Eval Accuracy: 81.42%


...

Epoch 28 | Eval Accuracy: 77.98%


Epoch 29/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 220.95it/s, Acc=98.7%, BER=0.00, L_Main=0.008, L_Wat=1.496]


Epoch 29 | Eval Accuracy: 79.59%


Epoch 30/30: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:04<00:00, 219.60it/s, Acc=98.7%, BER=0.00, L_Main=0.013, L_Wat=1.492]


Epoch 30 | Eval Accuracy: 79.70%

--- Embedding termine | BER final: 0.00 ---


In [4]:
# --- Test d'Extraction ---

print("=" * 50)
print("TEST D'EXTRACTION")
print("=" * 50)

# Extraction sur le modele watermarke
print("\n1. Extraction sur le modele watermarke:")
ber_watermarked, _ = greedy.extract(watermarked_model)


# Extraction sur un modele vierge (non-watermarke)
print("\n2. Extraction sur un modele vierge (devrait avoir BER ~0.5):")

model_id= "prajjwal1/bert-tiny"
config = BertConfig.from_pretrained(model_id, num_labels=2)
fresh_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device="cuda" if torch.cuda.is_available() else "cpu")
tokenizer=BertTokenizer.from_pretrained(model_id)

ber_fresh, _ = greedy.extract(fresh_model)


print(f"\nResume:")
print(f"  - BER modele watermarke: {ber_watermarked:.2f} (attendu: ~0.0)")
print(f"  - BER modele vierge: {ber_fresh:.2f} (attendu: ~0.5)")

TEST D'EXTRACTION

1. Extraction sur le modele watermarke:

2. Extraction sur un modele vierge (devrait avoir BER ~0.5):


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2807.34it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor


Resume:
  - BER modele watermarke: 0.00 (attendu: ~0.0)
  - BER modele vierge: 0.69 (attendu: ~0.5)


In [7]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack(greedy_obj, train_dataloader, eval_dataloader, epochs=5, lr=5e-5):
    """
    Tente de transferer la fonctionnalite du modele watermarke vers un modele vierge.
    Verifie si la marque (basee sur les poids) survit.
    """
    device = greedy_obj.device
    
    # 1. Teacher (Gele)
    teacher_model = greedy_obj.saved_keys["watermarked_model"]
    teacher_model.eval()
    for p in teacher_model.parameters():
        p.requires_grad = False
    
    # 2. Student (Vierge - Meme architecture)
    config = BertConfig.from_pretrained(model_id, num_labels=2)
    print("\n--- Initialisation du Student ---")
    student_model = BertForSequenceClassification.from_pretrained(model_id, config=config).to(device)
    student_model.train()
    
    # Verification initiale
    teacher_ber, _ = greedy_obj.extract(teacher_model)
    student_ber, _ = greedy_obj.extract(student_model)
    print(f"[Check] BER Teacher: {teacher_ber:.2f}")
    print(f"[Check] BER Student (Avant): {student_ber:.2f}")
    
    optimizer = torch.optim.AdamW(student_model.parameters(), lr=lr)
    history = {"loss": [], "ber": [], "acc": []}
    
    print(f"\n--- Distillation ({epochs} epochs) ---")
    feature_criterion = nn.MSELoss()
    for epoch in range(epochs):
        pbar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0
        
        for batch in pbar:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            
            optimizer.zero_grad()

            teacher_layer = greedy._get_target_layer(teacher_model, greedy.config["hook_layer"])
            hook = FeatureHook(teacher_layer)
            
            # Output Matching (Soft Labels)
            with torch.no_grad():
                teacher_logits = teacher_model(input_ids=input_ids, attention_mask=attention_mask).logits
                teacher_probs = F.softmax(teacher_logits / 2.0, dim=-1)  # Temperature=2
                t_features = hook.features
            hook.close()

            student_layer = greedy._get_target_layer(student_model, greedy.config["hook_layer"])
            hook = FeatureHook(student_layer)
            student_logits = student_model(input_ids=input_ids, attention_mask=attention_mask).logits
            student_log_probs = F.log_softmax(student_logits / 2.0, dim=-1)
            s_features = hook.features
            hook.close()


            loss_feat = feature_criterion(s_features.mean(dim=[0, 1]), t_features.mean(dim=[0, 1])) + feature_criterion(s_features.var(dim=[0, 1]), t_features.var(dim=[0, 1]))


            # KL Divergence Loss
            loss_kl = F.kl_div(student_log_probs, teacher_probs, reduction="batchmean") * (2.0 ** 2)
            loss= loss_kl +loss_feat
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            pbar.set_postfix(Loss=f"{loss.item():.4f}")
        
        # Evaluation
        avg_loss = running_loss / len(train_dataloader)
        history["loss"].append(avg_loss)
        
        current_ber, pred_wm_prob = greedy_obj.extract(student_model)
        history["ber"].append(current_ber)
        
        eval_acc = greedy_obj.evaluate(student_model, eval_dataloader)
        history["acc"].append(eval_acc)
        
        print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | BER: {current_ber:.2f} | Acc: {eval_acc:.1f}% err_wat: {nn.BCELoss()(pred_wm_prob, greedy_obj.saved_keys["watermark"]).item()}")
    
    return student_model, history

In [8]:
# --- Lancer l'attaque par distillation ---

student_model, attack_history = run_distillation_attack(
    greedy,
    train_dataloader, 
    eval_dataloader,
    epochs= 1000, # Plus d'epochs possible car modele rapide
)


--- Initialisation du Student ---


Loading weights: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 2221.89it/s, Materializing param=bert.pooler.dense.weight]
BertForSequenceClassification LOAD REPORT from: prajjwal1/bert-tiny
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
bert.embeddings.position_ids               | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
cls.predictions.decoder.bias               | UNEXPECTED | 
cls.predictions.decoder.weight             | UNEXPECTED | 
cls.predictions.transform.LayerNor

[Check] BER Teacher: 0.00
[Check] BER Student (Avant): 0.69

--- Distillation (1000 epochs) ---


Epoch 1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 184.17it/s, Loss=0.4751]


Epoch 1 | Loss: 1.3879 | BER: 0.56 | Acc: 80.6% err_wat: 56.25


...

Epoch 254 | Loss: 0.0081 | BER: 0.00 | Acc: 78.3% err_wat: 0.0


Epoch 255: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1053/1053 [00:05<00:00, 180.51it/s, Loss=0.0312]


Epoch 255 | Loss: 0.0090 | BER: 0.00 | Acc: 78.0% err_wat: 0.0


Epoch 256:  26%|██████████████████████████████████████████████████████████████▎                                                                                                                                                                                   | 271/1053 [00:01<00:04, 181.18it/s, Loss=0.0027]


KeyboardInterrupt: 

In [ ]:
# --- Visualisation des Resultats ---

import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(attack_history["loss"], 'b-o')
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Distillation Loss")
axes[0].set_title("Loss durant Distillation")
axes[0].grid(True)

# BER
axes[1].plot(attack_history["ber"], 'r-o')
axes[1].axhline(y=0.5, color='gray', linestyle='--', label='Random (0.5)')
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("BER")
axes[1].set_title("BER du Student (Watermark)")
axes[1].set_ylim([0, 1])
axes[1].legend()
axes[1].grid(True)

# Accuracy
axes[2].plot(attack_history["acc"], 'g-o')
axes[2].set_xlabel("Epoch")
axes[2].set_ylabel("Accuracy (%)")
axes[2].set_title("Accuracy du Student")
axes[2].grid(True)

plt.tight_layout()
plt.show()

# Resume final
print("\n" + "=" * 50)
print("RESUME FINAL")
print("=" * 50)
print(f"BER Teacher (watermarke): {greedy.extract(greedy.saved_keys['watermarked_model'])[0]:.2f}")
print(f"BER Student (apres distillation): {attack_history['ber'][-1]:.2f}")
print(f"Accuracy Student: {attack_history['acc'][-1]:.1f}%")

if attack_history['ber'][-1] > 0.4:
    print("\n-> La marque Greedy n'a PAS survecu a la distillation (BER eleve)")
else:
    print("\n-> La marque Greedy a survecu a la distillation (BER faible)")